In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import io
import numpy as np
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:
        try:
            result = {}
            result['url'] = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
            lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
            if lastmod is not None:
                result['lastmod'] = lastmod.text

            #title = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:title').text
            #keywords = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:keywords').text
            #news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))
            news_list.append(result)
        except:
            continue

    return news_list

In [3]:
sitemaps = parse_sitemap(io.StringIO(req.get('https://tass.ru/sitemap.xml').content.decode()))
len(sitemaps)

53

In [4]:
news_db = []

for fn in tqdm.tqdm_notebook(sitemaps):
    s1 = time.time()

    try:
        text = io.StringIO(req.get(fn['url']).content.decode())
        news = parse_sitemap(text)
    except Exception as e:
        print(e)
        continue
        
    news_db += news
    s2 = time.time()
    time.sleep(max(1 - s2 + s1, 0))

<ipython-input-4-1b27f3d996d8>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fn in tqdm.tqdm_notebook(sitemaps):


In [5]:
df = pd.DataFrame(news_db)
df['lastmod'] = pd.to_datetime(df['lastmod'], format='%Y-%m-%dT%H:%M:%S', utc=True)
df.head()

,url,lastmod
0,https://tass.ru/sport/10242051,2020-12-12 13:15:26+00:00
1,https://tass.ru/sport/10242087,2020-12-12 13:20:05+00:00
2,https://tass.ru/mezhdunarodnaya-panorama/10241989,2020-12-12 13:22:30+00:00
3,https://tass.ru/mezhdunarodnaya-panorama/10242079,2020-12-12 13:25:48+00:00
4,https://tass.ru/sport/10242125,2020-12-12 13:29:54+00:00


In [6]:
df[(df.lastmod >= '2019-11-01')].to_json('news_tass.json', date_unit='s', date_format='iso')
df[(df.lastmod >= '2019-11-01')].shape

(257124, 2)

# Parsing

In [19]:
def parse_news(text):
    soup = BeautifulSoup(text, features='lxml')
    
    news_text = ''
    lead = soup.find(name='div', attrs={'class': 'news-header__lead'})
    
    if lead:
        news_text += lead.get_text(' ', strip=True)

    news_body = soup.find(name='div', attrs={'class': 'text-content'})
    if news_body:
        news_ps = news_body.find_all(name='p')
        if news_ps:
            news_text += " " + " ".join([news_p.get_text(" ", strip=True) for news_p in news_ps])
    
    news_text = re.sub(r'[\n\r]+', ' ', news_text)

    tags_elems = soup.find_all(name="a", attrs={'class': "tags__item"})
    if tags_elems:
        tags = [re.sub(r'[\n\r]+', ' ', tag.get_text(" ", strip=True)) for tag in tags_elems]
        tags = ", ".join(tags)
    else:
        tags = ''
    
    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+', ' ', description_elem.get('content')).replace(u'\xa0', u' ') if description_elem else ''
        
    return {'news_text': news_text, 'tags': tags, 'description': description}

In [20]:
parse_news(req.get('https://tass.ru/nacionalnye-proekty/6528159').content)

{'news_text': 'Как сообщила главный внештатный терапевт ведомства Оксана Драпкина, речь идет о первом этапе диспансеризации, куда включен и онкоскрининг САНКТ-ПЕТЕРБУРГ, 8 июня. /ТАСС/. Большинство регионов уже готовы проводить первый этап диспансеризации, включающий профилактический осмотр, за один день, сообщила в субботу на полях Петербургского международного экономического форума главный внештатный терапевт Минздрава России Оксана Драпкина порталу "Будущее России. Национальные проекты" , оператором которого является ТАСС. Ранее Минздрав РФ опубликовал на федеральном портале проектов нормативных правовых актов проект приказа, дополняющий порядок прохождения профилактических осмотров и диспансеризации сроком в один день. "Мы говорим про первый этап диспансеризации, туда включен <…> и онкоскрининг. Реально все сдать, прийти и сделать все исследования в рамках первого этапа диспансеризации - это реально. По всем субъектам ситуация разная, но в принципе, если наладить и быть уверенным, 